<a href="https://colab.research.google.com/github/GabrielaGuestory/MachineLearning/blob/main/STRIDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Bloco 1 - Instalação de dependências
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow gradio openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
# Bloco 2 - Imports e Configuração do Azure OpenAI
import gradio as gr
import openai
import pytesseract
from PIL import Image

In [3]:
# Configure suas credenciais Azure OpenAI
openai.api_type = "azure"
openai.api_key = "SUA_CHAVE_AQUI"
openai.api_base = "https://SEU-ENDPOINT.openai.azure.com/"
openai.api_version = "2024-05-01-preview"  # ajuste conforme disponível

In [4]:
# Bloco 3 - Função de OCR + Análise STRIDE
def analyze_architecture(image_path):
    try:
        # Extrair texto do diagrama
        img = Image.open(image_path)
        extracted_text = pytesseract.image_to_string(img)

        if not extracted_text.strip():
            return "⚠️ Não foi possível extrair texto da imagem. Tente enviar uma imagem mais nítida."

        # Prompt STRIDE baseado no OCR
        prompt = f"""
        Você é um especialista em segurança de aplicações.
        Baseado no seguinte diagrama de arquitetura (texto extraído por OCR):
        ---
        {extracted_text}
        ---
        Gere uma análise de ameaças seguindo a metodologia STRIDE:
        - Spoofing
        - Tampering
        - Repudiation
        - Information Disclosure
        - Denial of Service
        - Elevation of Privilege
        Forneça a saída em formato claro e estruturado.
        """

        # Chamada ao modelo Azure OpenAI
        response = openai.ChatCompletion.create(
            engine="gpt-4",  # use o modelo que você tiver disponível
            messages=[
                {"role": "system", "content": "Você é um assistente especializado em segurança de aplicações."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=800
        )

        return response["choices"][0]["message"]["content"]

    except Exception as e:
        return f"Erro: {str(e)}"

In [5]:
# Bloco 4 - Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🔒 Análise de Ameaças STRIDE com Azure OpenAI (via OCR)")
    with gr.Row():
        image_input = gr.Image(type="filepath", label="Upload do Diagrama de Arquitetura")
    output = gr.Textbox(label="Relatório STRIDE", lines=15)
    submit = gr.Button("Analisar")

    submit.click(fn=analyze_architecture, inputs=image_input, outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd73eecaf18f6793bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
